In [50]:
from music21 import *
import keras
from keras import backend as k
import glob
import numpy
from keras.utils import np_utils 
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,LSTM
from keras.optimizers import RMSprop



In [36]:
!pip install librosa
import librosa

  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201381 sha256=b3242d17782cf9bcc860a0dd4969da2e21d168f407f9e75e36c360cc5345f8e0
  Stored in directory: c:\users\aakash\appdata\local\pip\cache\wheels\de\1e\aa\d91797ae7e1ce11853ee100bee9d1781ae9d750e7458c95afb
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23145 sha256=50a58d54f86894e2675e37c7bd2016befaa0acd8e6ec5ad58b986f764a99c3bd
  Stored in directory: c:\users\aakash\appdata\local\pip\cache\wheels\ba\7b\eb\213741ccc0678f63e346ab8dff10495995ca3f426af87b8d88
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320723 sha256=3d6aa49baf2bae755464ff05c1912f7c1f6a35b8a666c19c38cc32297c488e92
  Stored in directory: c:\users\aakash\appdata\local\pip\cache\wheels\a0\18\0a\8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
Successfully built librosa audioread resampy


In [47]:
!pip install --upgrade music21
import music21

  Created wheel for music21: filename=music21-6.3.0-py3-none-any.whl size=21888026 sha256=8d41bd3c4e928c965f666b25e3ced1d60b31175cda5c1a08c22825f390e449ea
  Stored in directory: c:\users\aakash\appdata\local\pip\cache\wheels\6a\d0\ae\11a9d9a4052ed2094afc14daf1f5a39569890dedb95be40612
Successfully built music21
  Attempting uninstall: music21
    Found existing installation: music21 6.1.0
    Uninstalling music21-6.1.0:
      Successfully uninstalled music21-6.1.0


In [52]:
notes = []
counter = 0
for file in glob.glob("midi_songs/*.mid"):
    print('processing song no: ',counter, file)
    counter = counter + 1
    file.encode('utf-8').strip()
    try:
        midi = converter.parse(file)
    except:
        continue
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))


processing song no:  0 midi_songs\0fithos.mid
processing song no:  1 midi_songs\8.mid
processing song no:  2 midi_songs\ahead_on_our_way_piano.mid
processing song no:  3 midi_songs\AT.mid
processing song no:  4 midi_songs\balamb.mid
processing song no:  5 midi_songs\bcm.mid
processing song no:  6 midi_songs\BlueStone_LastDungeon.mid
processing song no:  7 midi_songs\braska.mid
processing song no:  8 midi_songs\caitsith.mid
processing song no:  9 midi_songs\Cids.mid
processing song no:  10 midi_songs\cosmo.mid
processing song no:  11 midi_songs\costadsol.mid
processing song no:  12 midi_songs\dayafter.mid
processing song no:  13 midi_songs\decisive.mid
processing song no:  14 midi_songs\dontbeafraid.mid
processing song no:  15 midi_songs\DOS.mid
processing song no:  16 midi_songs\electric_de_chocobo.mid
processing song no:  17 midi_songs\Eternal_Harvest.mid
processing song no:  18 midi_songs\EyesOnMePiano.mid
processing song no:  19 midi_songs\ff11_awakening_piano.mid
processing song no

In [53]:
sequence_length = 100
n_vocab = len(set(notes))

# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab) 
network_output = np_utils.to_categorical(network_output)

In [54]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation

In [55]:
print(network_input.shape[1], network_input.shape[2])
print(len(network_input))

100 1
5369


In [56]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')





In [57]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 100, 256)          264192    
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 100, 512)          1574912   
_________________________________________________________________
dropout_22 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_14 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_23 (Dropout)         (None, 256)              

In [60]:

model.fit(network_input, network_output, epochs=50, batch_size=512)

Epoch 1/50
11/11 [==============================] - 130s 12s/step - loss: 4.6819
Epoch 2/50
11/11 [==============================] - 167s 15s/step - loss: 4.4318
Epoch 3/50
11/11 [==============================] - 105s 10s/step - loss: 4.4152
Epoch 4/50
11/11 [==============================] - 120s 11s/step - loss: 4.4039
Epoch 5/50
11/11 [==============================] - 138s 13s/step - loss: 4.3985
Epoch 6/50
11/11 [==============================] - 97s 9s/step - loss: 4.3881
Epoch 7/50
11/11 [==============================] - 79s 7s/step - loss: 4.3877
Epoch 8/50
11/11 [==============================] - 83s 8s/step - loss: 4.3763
Epoch 9/50
11/11 [==============================] - 77s 7s/step - loss: 4.3814
Epoch 10/50
11/11 [==============================] - 80s 7s/step - loss: 4.3688
Epoch 11/50
11/11 [==============================] - 81s 7s/step - loss: 4.3741
Epoch 12/50
11/11 [==============================] - 84s 8s/step - loss: 4.3629
Epoch 13/50
11/11 [====================

In [64]:
model_json = model.to_json()
with open("./model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [65]:
# now we will generate the music
# model = Sequential()
# model.add(LSTM(
#     512,
#     input_shape=(network_input.shape[1], network_input.shape[2]),
#     return_sequences=True
# ))
# model.add(Dropout(0.3))
# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(512))
# model.add(Dense(256))
# model.add(Dropout(0.3))
# model.add(Dense(n_vocab))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# # Load the weights to each node
# model.load_weights('weights-improvement-99-0.2833-bigger.hdf5')


model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('model.h5')

In [66]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]

prediction_output = []
# generate 500 notes
for note_index in range(200):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    ind =numpy.asarray([index])
    pattern = numpy.append(pattern, ind)
    pattern = pattern[1:len(pattern)]

In [67]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [68]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

In [1]:

import pygame

def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print("Music file %s loaded!" % music_file)
    except pygame.error:
        print("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
# pick a midi music file you have ...
# (if not in working folder use full path)



pygame 2.0.0 (SDL 2.0.12, python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
midi_file = 'test_output.mid'
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)

try:
    play_music(midi_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

Music file test_output.mid loaded!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-5-f972e23f3b9b>", line 9, in <module>
    play_music(midi_file)
  File "<ipython-input-1-9024338d1fa5>", line 18, in play_music
    clock.tick(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-f972e23f3b9b>", line 15, in <module>
    raise SystemExit
SystemExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 319, in wrapped
    return f(*args, **kwargs)
  

SystemExit: 

In [69]:
# playing the file
mf = midi.MidiFile()
mf.open('test_output.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')

AttributeError: 'Score' object has no attribute 'MidiFile'